In [ ]:
import pandas as pd
from hnmchallenge.data_reader import DataReader
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np



from hnmchallenge.models.top_pop import TopPop
from hnmchallenge.evaluation.python_evaluation import map_at_k
from hnmchallenge.constant import *

In [ ]:
import logging
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

In [ ]:
dataset = FilterdDataset()
dr = DataReader()

In [4]:
fd = dr.get_filtered_full_data()

In [ ]:
fd

In [ ]:
fd[DEFAULT_USER_COL].nunique()

In [ ]:
fd = fd.drop_duplicates([DEFAULT_USER_COL, DEFAULT_ITEM_COL])

In [ ]:
fd

In [ ]:
count_mb = fd.groupby(DEFAULT_ITEM_COL).count()
feature = count_mb.reset_index()[[DEFAULT_ITEM_COL, "t_dat"]].rename(
            columns={"t_dat": "popularity"})

In [ ]:
feature

In [ ]:
feature["popularity_score"]=(feature["popularity"]-feature["popularity"].min())/(feature["popularity"].max()-feature["popularity"].min())

In [ ]:
feature

In [ ]:
feature['rank'] = feature['popularity_score'].rank(ascending=False,method='min').astype(int)

In [ ]:
feature

In [ ]:
df=dr.get_filtered_full_data()

In [ ]:
df= df.drop_duplicates([DEFAULT_USER_COL, DEFAULT_ITEM_COL])

In [ ]:
df

In [ ]:
df["last_buy"] = df.groupby(DEFAULT_USER_COL)["t_dat"].transform(max)
df["first_buy"] = df.groupby(DEFAULT_USER_COL)["t_dat"].transform(min)
df["time_score"] = (df["t_dat"] - df["first_buy"]) / (
            df["last_buy"] - df["first_buy"]
        )

In [ ]:
df=df.fillna(1)#df["tdiff"] =1/ ((df["last_buy"]-df["t_dat"]).dt.days) 

In [ ]:
df

In [ ]:
df["rank_time"] = df.groupby(DEFAULT_USER_COL)["time_score"].rank(ascending=False,method='min').astype(int)

In [ ]:
df

In [ ]:
df.loc[df[DEFAULT_USER_COL]==1]

In [ ]:
feature

In [ ]:
final=pd.merge(df, feature, on =DEFAULT_ITEM_COL, how='left')

In [ ]:
final

In [ ]:
final.loc[final[DEFAULT_USER_COL]==1]

In [ ]:
user=dr.get_filtered_full_data()

In [ ]:
user = user.drop_duplicates([DEFAULT_USER_COL])

In [ ]:
user['temp']=1

In [ ]:
user=user[[DEFAULT_USER_COL,'temp']]

In [ ]:
user

In [ ]:
feature_100=feature[feature['rank']<=100]

In [ ]:
feature_100['temp']=1

In [ ]:
final1=pd.merge(user, feature_100, on='temp')

In [ ]:
final1

In [ ]:
final1=final1.drop('temp', axis=1)

In [ ]:
final1

In [ ]:
final1[final1['customer_id']==1]

In [ ]:
final

In [ ]:
final=final.drop(['popularity_score','rank','popularity','t_dat','price','sales_channel_id','last_buy','first_buy'], axis=1)

In [ ]:
final2=pd.merge(final,final1,on=[DEFAULT_USER_COL,DEFAULT_ITEM_COL],how='outer')

In [ ]:
final[final[DEFAULT_USER_COL]==1]

In [ ]:
final2[final2[DEFAULT_USER_COL]==1]

In [ ]:
mask = final2.isnull().any(axis=1)

In [ ]:
final3=final2[~mask]

In [ ]:
final3[final3[DEFAULT_USER_COL]==1]

In [ ]:
final2=final2.fillna(0)

In [ ]:
final2

In [ ]:
final2=final2.drop(['rank_time','rank'],axis=1)

In [ ]:
a=0.95
eps=1e-6
final2['score']=a*final2['time_score']+(1-a+eps)*final2['popularity_score']

In [ ]:
final2

In [ ]:
final2["rank"] = final2.groupby(DEFAULT_USER_COL)["score"].rank(ascending=False,method='min').astype(int)

In [ ]:
final2

In [ ]:
final2=final2.drop(["time_score","popularity","popularity_score"],axis=1)

In [ ]:
final2[DEFAULT_USER_COL].nunique()

In [ ]:
final2

In [ ]:
cutoff = final2.groupby(DEFAULT_USER_COL).size().values

In [ ]:
cutoff

In [ ]:
i=0
filter_indices = []
for cut in cutoff:
    filter_indices.extend(range(i, i+12))
    i=i+cut

In [ ]:
filter_indices

In [ ]:
final2.head()

In [ ]:
final2 = final2.sort_values(DEFAULT_USER_COL, ascending=True)

In [ ]:
final2.head()

In [ ]:
final2 = final2.sort_values([DEFAULT_USER_COL, "rank"], ascending=True)

In [ ]:
final2

In [ ]:
final2 = final2.reset_index(drop=True)

In [ ]:
final_df = final2.loc[filter_indices]

In [ ]:
final_df[DEFAULT_USER_COL].nunique()

In [ ]:
final_final_df = final_df.drop(["score","rank"] ,axis=1)

In [ ]:
final_final_df[DEFAULT_USER_COL].nunique()

In [ ]:
from hnmchallenge.submission_handler import SubmissionHandler
sh = SubmissionHandler()     

In [ ]:
sh.create_submission_filtered_data([final_final_df], sub_name="Time_Pop")